In [1]:
import keras
import numpy as np
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Conv2D, Dense, Input, Dropout, Flatten, Activation, MaxPooling2D, AveragePooling2D, BatchNormalization
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD
from keras import backend as K
from keras.utils.np_utils import to_categorical  
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
import json
K.set_image_dim_ordering('tf') 

Using TensorFlow backend.


In [20]:
train_img_size = (64,64) #scaling down images to 64x64

In [39]:
f = pd.read_csv('training.csv') # Reading training.csv
all_rows = []
print('image bounding box data construction starting...')
train_data = pd.DataFrame([],columns=['height','img_name','label','left','top','width','img_width','img_height'])
for index, values in f.iterrows():
    img_name = values['image_name']
    row_dict = {}
    row_dict['left'] = values['x1']
    row_dict['top'] = values['y1']
    row_dict['height'] = values['y2']-values['y1']
    row_dict['width'] = values['x2']-values['x1']
    row_dict['label'] = 0
    row_dict['img_name'] = img_name
    img1=cv2.imread(os.path.join('../../images',img_name))
    row_dict['img_height']=img1.shape[0]
    row_dict['img_width']=img1.shape[1]
    all_rows.append(row_dict)
    train_data = pd.concat([train_data,pd.DataFrame(row_dict,index=[index])])
    print(str(index)+'/'+str(f.shape[0]))
    
train_data['bottom'] = train_data['top']+train_data['height']
train_data['right'] = train_data['left']+train_data['width']

#Scaling down the coordinates of bounding box according to scaled down image
train_data['top'] = train_data['top']*(train_img_size[0]/train_data['img_height'])
train_data['left'] = train_data['left']*(train_img_size[1]/train_data['img_width'])
train_data['bottom'] = train_data['bottom']*(train_img_size[0]/train_data['img_height'])
train_data['right'] = train_data['right']*(train_img_size[1]/train_data['img_width'])
train_data['width'] = train_data['right'] - train_data['left']
train_data['height'] = train_data['bottom'] - train_data['top']

image bounding box data construction starting...
0/14000
1/14000
2/14000
3/14000
4/14000
5/14000
6/14000
7/14000
8/14000
9/14000
10/14000
11/14000
12/14000
13/14000
14/14000

C:\Users\Devansh\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




15/14000
16/14000
17/14000
18/14000
19/14000
20/14000
21/14000
22/14000
23/14000
24/14000
25/14000
26/14000
27/14000
28/14000
29/14000
30/14000
31/14000
32/14000
33/14000
34/14000
35/14000
36/14000
37/14000
38/14000
39/14000
40/14000
41/14000
42/14000
43/14000
44/14000
45/14000
46/14000
47/14000
48/14000
49/14000
50/14000
51/14000
52/14000
53/14000
54/14000
55/14000
56/14000
57/14000
58/14000
59/14000
60/14000
61/14000
62/14000
63/14000
64/14000
65/14000
66/14000
67/14000
68/14000
69/14000
70/14000
71/14000
72/14000
73/14000
74/14000
75/14000
76/14000
77/14000
78/14000
79/14000
80/14000
81/14000
82/14000
83/14000
84/14000
85/14000
86/14000
87/14000
88/14000
89/14000
90/14000
91/14000
92/14000
93/14000
94/14000
95/14000
96/14000
97/14000
98/14000
99/14000


In [40]:
# Reading the images from the dataset and storing them in pandas series
x = []

final = train_data
for index, values in final.iterrows():
    img_name = values['img_name']
    img2=np.array(cv2.imread(os.path.join('../../images',img_name)))
    #print(img2)
    x.append(img2)
    
    if index%50==0:
        print(index)
images=pd.Series(x)

0
50
100


,height,img_height,img_name,img_width,label,left,top,width,bottom,right
0,24.2667,480,JPEG_20160706_121146_1000145715002.png,640,0,11.5,19.0667,38,43.3333,49.5
1,46.4,480,JPEG_20161119_174038_1000690577600.png,640,0,2.3,8.13333,43.4,54.5333,45.7
2,60.9333,480,147444927651111470309333776-Roadster-Men-Casua...,640,0,3.7,1.73333,56.4,62.6667,60.1
3,29.7333,480,147772332675720161028_161611.png,640,0,2.7,21.6,57.5,51.3333,60.2
4,20.1333,480,1473315333651DeeplearnS11638.png,640,0,6,23.2,52.6,43.3333,58.6
5,54,480,JPEG_20161129_143914_1000304878557.png,640,0,3.5,4.93333,51.4,58.9333,54.9
6,46.4,480,JPEG_20161028_100621_1000815239330.png,640,0,7.7,11.4667,48.1,57.8667,55.8
7,30,480,1475060075141DSC02381.png,640,0,5,12.6667,57.7,42.6667,62.7
8,46,480,1473317734429DeeplearnS142.png,640,0,10.4,11.0667,44.8,57.0667,55.2
9,48.2667,480,JPEG_20160607_153530_1000669241198.png,640,0,13.4,8.93333,36.6,57.2,50


In [41]:
#Changing the image from color to gray-scale and resizing it to 64X64
train_labels = np.array(train_data[['top','left','width','height']])
tmp = []
train_images = images
for index,img in train_images.items():
    tmp.append(cv2.normalize(cv2.cvtColor(cv2.resize(img,train_img_size), cv2.COLOR_BGR2GRAY).astype(np.float64), 0, 1, cv2.NORM_MINMAX)[...,np.newaxis])
    print(index)
train_images = np.array(tmp)
single_img_shape = train_images[0].shape #Shape of image


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [42]:
train_labels = np.array(train_data[['top','left','width','height']]) #Train labels used

In [44]:
#ConvNet to train bounding box model
k_size = 3
cnn1 = Input(shape=single_img_shape)
#x = BatchNormalization()(cnn1)
x = Conv2D(32, k_size, k_size, activation='relu', border_mode='same')(cnn1)
x = BatchNormalization()(x)
x = Conv2D(64, k_size, k_size, activation='relu', border_mode='same')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(32, k_size, k_size, activation='relu', border_mode='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, k_size, k_size, activation='relu', border_mode='same')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Conv2D(128, k_size, k_size, activation='relu', border_mode='same')(x)
x = BatchNormalization()(x)
x = Conv2D(64, k_size, k_size, activation='relu', border_mode='same')(x)
x = BatchNormalization()(x)
# x = Dropout(0.2)(x)
x = Conv2D(128, k_size, k_size, activation='relu', border_mode='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, k_size, k_size, activation='relu', border_mode='same')(x)
x = BatchNormalization()(x)
x = Conv2D(128, k_size, k_size, activation='relu', border_mode='same')(x)
x = BatchNormalization()(x)
x = Conv2D(256, k_size, k_size, activation='relu', border_mode='same')(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(2048, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
h = Dense(4)(x)
model = Model(input=cnn1, output=h)
model.compile(loss='mse', optimizer='adadelta',metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=50, batch_size=128, validation_split=0.2, verbose=1)


C:\Users\Devansh\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  after removing the cwd from sys.path.
C:\Users\Devansh\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  
C:\Users\Devansh\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Devansh\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  if sys.path[0] == '':
C:\Users\Devansh\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D

Train on 80 samples, validate on 21 samples
Epoch 1/1
80/80 [==============================] - 176s 2s/step - loss: 994.8895 - acc: 0.2375 - val_loss: 4741.6504 - val_acc: 0.7143


In [87]:
#Reading the test images and adding labels:-image_width and image_height
f = pd.read_csv('test.csv') 
all_rows = []
print('image bounding box data construction starting...')
test_data = pd.DataFrame([],columns=['img_name','img_width','img_height'])
for index, values in f.iterrows():
    
    
    img_name = values['image_name']
    row_dict = {}
    row_dict['img_name'] = img_name
    img1=cv2.imread(os.path.join('../../images',img_name))
    print(index)
    row_dict['img_height']=img1.shape[0]
    row_dict['img_width']=img1.shape[1]
    all_rows.append(row_dict)
    test_data = pd.concat([test_data,pd.DataFrame(row_dict,index=[index])])

image bounding box data construction starting...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14

C:\Users\Devansh\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [49]:
#Reading the test images from the dataset and storing it in pandas series
x = []
final = pd.read_csv('test.csv')
for index, values in final.iterrows():
    img_name = values['image_name']
    img2=np.array(cv2.imread(os.path.join('../../images',img_name)))
    #print(img2)
    x.append(img2)
    if index%50==0:
        print(index)
images_test=pd.Series(x)

0
50


In [50]:
# Changing the test images from color to gray-scale and resizing it to 64X64
tmp = []
test_img_size = (64,64)
test_images = images_test
for index,img in test_images.items():
    tmp.append(cv2.normalize(cv2.cvtColor(cv2.resize(img,test_img_size), cv2.COLOR_BGR2GRAY).astype(np.float64), 0, 1, cv2.NORM_MINMAX)[...,np.newaxis])
    print(index)

test_images = np.array(tmp)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [51]:
# Predicting our ConvNet model
preds = model.predict(test_images)

In [54]:
# Converting predictions to a dataframe
preds1=pd.DataFrame(preds)
preds1

,0,1,2,3
0,-93.813988,-59.090038,53.662823,109.031456
1,-97.263771,-103.578568,124.808022,111.612679
2,-158.713089,-121.044701,130.319443,199.257263
3,-12.882331,3.539965,-1.066837,13.799304
4,-135.303497,-96.710579,94.353676,155.733047
5,-17.599813,-3.522770,1.944385,10.570508
6,0.446463,-6.145773,4.399351,-16.503536
7,-96.717918,-89.816734,85.026276,121.829254
8,-267.526428,-214.308014,240.642532,347.684875
9,-1.917684,-11.461452,12.464473,-0.097401


In [88]:
# Upsacling the predictions on test images
testfinal = pd.read_csv('test.csv')
avg=0
for index, img in preds1.iterrows():
    print(str(index)+'/'+str(testfinal.shape[0]))
    testfinal.loc[testfinal.index[index], 'x1'] = img[1]*test_data.loc[test_data.index[index], 'img_width']/64
    testfinal.loc[testfinal.index[index], 'y1'] = img[0]*test_data.loc[test_data.index[index], 'img_height']/64
    testfinal.loc[testfinal.index[index], 'x2'] = (img[1]+img[2])*test_data.loc[test_data.index[index], 'img_width']/64
    testfinal.loc[testfinal.index[index], 'y2'] = (img[0]+img[3])*test_data.loc[test_data.index[index], 'img_height']/64
    #     img['x1']=maxres['topleft']['x']
    #     img['y1']=maxres['topleft']['y']
    #     img['x2']=maxres['bottomright']['x']
    #     img['y2']=maxres['bottomright']['y']
# testfinal.drop(['Unnamed: 0'],axis=1,inplace=True)
testfinal.to_csv('test.csv',index=False)
testfinal

0/12815
1/12815
2/12815
3/12815
4/12815
5/12815
6/12815
7/12815
8/12815
9/12815
10/12815
11/12815
12/12815
13/12815
14/12815
15/12815
16/12815
17/12815
18/12815
19/12815
20/12815
21/12815
22/12815
23/12815
24/12815
25/12815
26/12815
27/12815
28/12815
29/12815
30/12815
31/12815
32/12815
33/12815
34/12815
35/12815
36/12815
37/12815
38/12815
39/12815
40/12815
41/12815
42/12815
43/12815
44/12815
45/12815
46/12815
47/12815
48/12815
49/12815
50/12815
51/12815
52/12815
53/12815
54/12815
55/12815
56/12815
57/12815
58/12815
59/12815
60/12815
61/12815
62/12815
63/12815
64/12815
65/12815
66/12815
67/12815
68/12815
69/12815
70/12815
71/12815
72/12815
73/12815
74/12815
75/12815
76/12815
77/12815
78/12815
79/12815
80/12815
81/12815
82/12815
83/12815
84/12815
85/12815
86/12815
87/12815
88/12815
89/12815
90/12815
91/12815
92/12815
93/12815
94/12815
95/12815
96/12815
97/12815
98/12815
99/12815
100/12815


,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,-590.900383,-54.272156,-703.604908,114.131012
1,1473231475010DeeplearnS11276.png,-1035.785675,212.294540,-729.478283,107.616806
2,JPEG_20161205_135307_1000155917326.png,-1210.447006,92.747421,-1190.348167,304.081306
3,JPEG_20160711_123440_1000518778437.png,35.399654,24.731283,-96.617482,6.877298
4,JPEG_20160803_115329_100034020722.png,-967.105789,-23.569031,-1014.776230,153.221626
5,147444974116511473239803010-Mast--Harbour-Men-...,-35.227704,-15.783855,-131.998601,-52.719791
6,JPEG_20160622_110649_1000527459853.png,-61.457729,-17.464218,3.348474,-120.428052
7,JPEG_20160823_120737_1000784898268.png,-898.167343,-47.904587,-725.384388,188.335018
8,1480965956645IMG_3094.png,-2143.080139,263.345184,-2006.448212,601.188354
9,14732348976417a565e40d545452688130062b267a2d2.png,-114.614525,10.030203,-14.382629,-15.113139
